In [ ]:
!pip install git+https://github.com/mementum/backtrader.git@0fa63ef4a35dc53cc7320813f8b15480c8f85517#egg=backtrader
!pip install yfinance
!pip install mplfinance
!pip install pandas-bokeh

In [ ]:
import math
import backtrader as bt
import yfinance as yf
import mplfinance as mpf
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['figure.figsize'] = [12, 6]

In [ ]:
from components.BuyAndHold import BuyAndHold
from components.DCA import DollarCostAverage
from components.VA import ValueInvesting
from components.utils import ComissionInfo, CashMarket

In [ ]:
def get_yf_data(ticker="BTC-USD", interval="1d", period="max"): 
    ticker_yf = yf.Ticker(ticker)
    hist = ticker_yf.history(period=period, interval=interval)
    hist.ffill(inplace=True)
    return bt.feeds.PandasData(dataname = hist)

data = get_yf_data()

In [ ]:
def test_strategy(data, strategy=None, initial_cash=1000):
    cerebro = bt.Cerebro()
    cerebro.adddata(data)
    cerebro.addstrategy(strategy)

    broker_args = dict(coc=True)
    cerebro.broker = bt.brokers.BackBroker(**broker_args)
    ankwargs = dict(timeframe=bt.TimeFrame.Years)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, fund=True, **ankwargs)
    cerebro.addanalyzer(CashMarket)
    cerebro.broker.addcommissioninfo(ComissionInfo())

    cerebro.broker.set_cash(initial_cash)
    results = cerebro.run()
#     cerebro.plot(iplot=False, style='candlestick')
    return results



In [ ]:
import seaborn as sns
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()

In [ ]:
tickers = {
    'BTC': get_yf_data(period="5y"),
    'SPY': get_yf_data(ticker="SPY", period="5y"),
    'VWRL': get_yf_data(ticker="VWRL.AS", period="5y")
}

strategies = {
    "Dollar Cost Average": DollarCostAverage,
    "Value Averaging": ValueInvesting
}

In [ ]:
import itertools
results_dfs = []
for data, strategy in itertools.product(tickers, strategies):
    print(f"Results For {strategy} on {data}")
    results = test_strategy(tickers[data], strategy=strategies[strategy])[0]
    value_analyser = results.analyzers[1].get_analysis()
    results_df = pd.DataFrame.from_dict(value_analyser, orient='index', columns=[f"{data}-{strategy}"])
    results_dfs.append(results_df)

df = pd.concat(results_dfs, axis=1)   
df.ffill(inplace=True)

In [ ]:
df

In [68]:
df.plot_bokeh(
    figsize=(1200, 650),
    logy=True,
    legend="top_left"
)

Figure(id='2377', ...)

In [ ]:
tickers = {
    'BTC': get_yf_data(period="1y"),
    'SPY': get_yf_data(ticker="SPY", period="1y"),
    'VWRL': get_yf_data(ticker="VWRL.AS", period="1y")
}

strategies = {
    "Dollar Cost Average": DollarCostAverage,
    "Value Averaging": ValueInvesting
}

In [ ]:
import itertools
results_dfs = []
for data, strategy in itertools.product(tickers, strategies):
    print(f"Results For {strategy} on {data}")
    results = test_strategy(tickers[data], strategy=strategies[strategy])[0]
    value_analyser = results.analyzers[1].get_analysis()
    results_df = pd.DataFrame.from_dict(value_analyser, orient='index', columns=[f"{data}-{strategy}"])
    results_dfs.append(results_df)

df = pd.concat(results_dfs, axis=1)   
df.ffill(inplace=True)

In [ ]:
df.plot_bokeh(
    figsize=(1200, 650),
    logy=True,
    legend="top_left"
)